In [ ]:
import os
import numpy as np
import pandas as pd
import torch
from torch import Tensor
import torch.nn.functional as F

!pip install torch_geometric
!pip uninstall torch-scatter torch-sparse torch-geometric torch-cluster  --y
!pip install torch-scatter -f https://data.pyg.org/whl/torch-{torch.__version__}.html
!pip install torch-sparse -f https://data.pyg.org/whl/torch-{torch.__version__}.html
!pip install torch-cluster -f https://data.pyg.org/whl/torch-{torch.__version__}.html
!pip install git+https://github.com/pyg-team/pytorch_geometric.git
!pip install torchviz
!pip install matplotlib

import torch_geometric.transforms as T
from torch_geometric.data import HeteroData
from torch_geometric.loader import LinkNeighborLoader
from torch_geometric.nn import SAGEConv, to_hetero


In [ ]:
!git clone https://github.com/Synerise/predicting-user-behavior-workshop.git
DATA_DIR = "./predicting-user-behavior-workshop/data"

In [ ]:
TRAIN_PD_PATH = f"{DATA_DIR}/2019-Oct_small.csv"
TARGETS_PD_PATH = f"{DATA_DIR}/2019-Nov_train_target.csv"
TEST_TARGETS_PATH = f"{DATA_DIR}/test_target.npy"
TARGET_LABELS = f'{DATA_DIR}/target_brands.npy'
USER_IDS = f'{DATA_DIR}/user_ids.npy'

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: '{device}'")

In [ ]:
def load_data():
    train_df = pd.read_csv(TRAIN_PD_PATH)
    target_df = pd.read_csv(TARGETS_PD_PATH)
    test_targets_np = np.load(TEST_TARGETS_PATH)
    target_labels = np.load(TARGET_LABELS, allow_pickle=True)
    user_ids = np.load(USER_IDS)

    return train_df, target_df, test_targets_np, target_labels, user_ids

In [ ]:
train_df, target_df, targets, target_labels, users_in_target = load_data()

In [ ]:
train_df

In [ ]:
target_df = target_df[target_df['user_id'].isin(train_df['user_id'].unique())]

In [ ]:
train_df = pd.concat([train_df, target_df])

# Create artificial IDs for users and brands

In a few moments we'll be using the PyG library. We must create some objects required by the library to represent our graph. First, we will map users and brands to artificial integer IDs.

In [ ]:
users_in_target.shape, train_df['user_id'].nunique()

In [ ]:
unique_user_ids = pd.DataFrame(data={
    'user_id': users_in_target,
    'mapped_user_id': pd.RangeIndex(len(users_in_target)),
})
print("Mapping of user IDs to consecutive values:")
print("==========================================")
print(unique_user_ids.head())
print()

In [ ]:
unique_brands = train_df['brand'].unique()
unique_brands = pd.DataFrame(data={
    'brand': unique_brands,
    'mapped_brand': pd.RangeIndex(len(unique_brands)),
})
print("Mapping of brands to consecutive values:")
print("===========================================")
print(unique_brands)

Next, we merge the IDs with our dataset:

In [ ]:
purchases_user_id = pd.merge(train_df['user_id'], unique_user_ids,
                            left_on='user_id', right_on='user_id', how='left')
purchases_brand = pd.merge(train_df['brand'], unique_brands,
                            left_on='brand', right_on='brand', how='left')

We switch to representing our graph by only the artificial IDs, by taking matching user ID and brand ID of each transaction:

In [ ]:
purchases_user_id = torch.from_numpy(purchases_user_id['mapped_user_id'].values)
purchases_brand = torch.from_numpy(purchases_brand['mapped_brand'].values)

# Create the HeteroData object

Now, we finally create the native PyG object representing our graph. It's called HeteroData because we have a few different node types inside the graph. First, we merge the matching user and brand ids into a 2d numpy array:

In [ ]:
edge_index_user_to_brand = torch.stack([purchases_user_id, purchases_brand], dim=0)

print("Final edge indices pointing from users to brands:")
print("=================================================")
print(edge_index_user_to_brand)

Then, we create an undirected HeteroData object.
We need to name the main relationship between **users** and **brands**. We call it **buys** to reflect the relationship "user A buys brand B".

In [ ]:
num_users = len(unique_user_ids)
num_brands = len(unique_brands)

data = HeteroData()
data["user"].node_id = torch.arange(num_users)
data["brand"].node_id = torch.arange(num_brands)

#give some name to the relationship between user nodes and brand nodes
data["user", "buys", "brand"].edge_index = edge_index_user_to_brand
data = T.ToUndirected()(data)

# Visualize the Graph with networkx

In [ ]:
import networkx as nx

subgraph = edge_index_user_to_brand[:, :200].T.numpy()

user2name = {x:f'user{x}' for x in subgraph[:, 0]}
brand2name = {x:f'brand{x}' for x in subgraph[:, 1]}

named_subgraph = [(user2name[start], brand2name[end]) for start, end in subgraph]

g = nx.from_edgelist(named_subgraph)
g = nx.DiGraph(g)

nx.draw(g, node_size=40)

In [ ]:
g.nodes

# Exercise 1

We want to color the nodes to show which are USERS, and which are BRANDS. Create a `color_map` list with strings: 'red' for users, and 'green' for brands, to match the ordering of USER and BRAND nodes in `g.nodes` above.

The result should look like this:
`color_map = ('green', 'red', 'green', 'red', ...)`

In [ ]:
#@title Solution

color_map = ['green' if x.startswith('brand') else 'red' for x in g.nodes]

In [ ]:
#BRANDS are GREEN
#USERS are RED
nx.draw(g, node_size=40, node_color=color_map)

#Define the Splits

Now, we need to define how our data will be split during training:



*   `num_val` - what portion of the graph we want to validate on during training
*   `num_test` - what portion of the graph we want to use for testing (it's 0.0 because we want to do a full prediction for all users during test)
* `disjoint_train_ratio=0.3` - 30% of edges will be used as training targets (0 or 1 values will teach the model whether it should predict an edge or not between a given pair of nodes). 70% of edges will be used for aggregating neighbors and walking over the graph.  
* `neg_sampling_ratio=2.0` - non-existent edges will be created as well (to be used as negative examples during validation) with a ratio of 2 non-existent edges to 1 existing edge
* `add_negative_train_samples=False` - negative edges during training will not be generated beforehand, but will be generated on-the-fly
* `edge_types` - our chosen name for the user-brand relationship
* `rev_edge_types` - we want to go from users to brands, but also from brands to users (two-way connections in the graph). We call it a *reverse* buy operation




In [ ]:
transform = T.RandomLinkSplit(
    num_val=0.1,
    num_test=0.0,
    disjoint_train_ratio=0.3,
    neg_sampling_ratio=2.0,
    add_negative_train_samples=False,
    edge_types=("user", "buys", "brand"),
    rev_edge_types=("brand", "rev_buys", "user"),
)
train_data, val_data, _ = transform(data)

In [ ]:
train_data

# Define the Loader

We must also create a dataloader to generate training batches of the data. Neural networks rarely accept the whole dataset at once. More often, they gradually update their weights using portions of data (batches) until the whole dataset is processed.

Some important parameters to give here:

* `num_neighbors` - select max number of first-order neighbors and second-order neighbors
* `neg_sampling_ratio` - similarly as before - define how many artificial negative edges should be generated for training.
* `batch_size` - how many nodes (with their neighborhoods) we want to accept into the neural network in a given time
* `shuffle=True` - we want to mix the ordering of training examples (it is shown to improve neural network quality)

In [ ]:
edge_label_index = train_data["user", "buys", "brand"].edge_label_index
edge_label = train_data["user", "buys", "brand"].edge_label
train_loader = LinkNeighborLoader(
    data=train_data,
    num_neighbors=[5, 5],
    neg_sampling_ratio=2.0,
    edge_label_index=(("user", "buys", "brand"), edge_label_index),
    edge_label=edge_label,
    batch_size=128,
    shuffle=True,
)

# Create the GNN Architecture

In [ ]:
class GNN(torch.nn.Module):
    def __init__(self, hidden_channels):
        """
        Defines the architecture of our GNN, such as the type and number of layers.
        """
        super().__init__()
        #create 2 GraphSAGE layers:
        self.conv1 = SAGEConv(hidden_channels, hidden_channels)
        self.conv2 = SAGEConv(hidden_channels, hidden_channels)
    def forward(self, x: Tensor, edge_index: Tensor) -> Tensor:
        x = F.relu(self.conv1(x, edge_index))
        x = self.conv2(x, edge_index)
        return x

class Model(torch.nn.Module):
    def __init__(self, hidden_channels):
        super().__init__()
        self.num_users = data["user"].num_nodes
        self.num_brands = data["brand"].num_nodes

        #create untrained Embedding objects for users and brands. These embeddings will start to change during network training.
        self.user_emb = torch.nn.Embedding(self.num_users, hidden_channels)
        self.brand_emb = torch.nn.Embedding(self.num_brands, hidden_channels)
        # Instantiate a homogeneous GNN:
        self.gnn = GNN(hidden_channels)
        # Convert GNN model into a heterogeneous variant:
        self.gnn = to_hetero(self.gnn, metadata=data.metadata())

    def forward(self, data: HeteroData) -> Tensor:
        edge_label_index = data["user", "buys", "brand"].edge_label_index

        x_dict = self.encode(data)
        pred = self.decode(x_dict["user"], x_dict["brand"], edge_label_index)
        return pred

    def encode(self, data: HeteroData) -> Tensor:
        """
        Takes a HeteroData object (so, our graph or a part of it) and runs it thought our GNN.
        We obtain a dictionary of node embeddings.
        """
        x_dict = {
          "user": self.user_emb(data["user"].node_id),
          "brand": self.brand_emb(data["brand"].node_id),
        }
        x_dict = self.gnn(x_dict, data.edge_index_dict)
        return x_dict

    def decode(self, x_user: Tensor, x_brand: Tensor, edge_label_index: np.array) -> Tensor:
        """
        Computes similarity between given user and brand embeddings.
        Similarity defines the "relatedness" of a given user with a given brand.
        """
        edge_feat_user = x_user[edge_label_index[0]]
        edge_feat_brand = x_brand[edge_label_index[1]]
        # Apply dot-product to get a prediction per supervision edge:
        return (edge_feat_user * edge_feat_brand).sum(dim=-1)

model = Model(hidden_channels=32)

In [ ]:
from torchviz import make_dot


batch = next(iter(train_loader))
y = model(batch)

make_dot(y.mean(), params=dict(model.named_parameters()))

# Training

We train the model on a **link prediction** task. The training objective is to predict whether there is an edge between given nodes.

In [ ]:
import tqdm
import torch.nn.functional as F

model = model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
for epoch in range(1, 2):
    total_loss = total_examples = 0
    for sampled_data in tqdm.tqdm(train_loader):
        optimizer.zero_grad()
        sampled_data.to(device)
        pred = model(sampled_data)
        ground_truth = sampled_data["user", "buys", "brand"].edge_label

        #this is a loss function for binary classification (0 = no edge between nodes exists, 1 = an edge between nodes exists)
        loss = F.binary_cross_entropy_with_logits(pred, ground_truth)
        loss.backward()
        optimizer.step()
        total_loss += float(loss) * pred.numel()
        total_examples += pred.numel()
    print(f"Epoch: {epoch}, Loss: {total_loss / total_examples:.4f}")

In [ ]:
brand_dict = dict(zip(unique_brands['brand'].values, unique_brands['mapped_brand'].values))
target_brand_ids = [brand_dict[brand] for brand in target_labels]

In [ ]:
target_brand_ids

# Testing

For testing, we need to create a new loader with no negative edges. We want to gather the trained embeddings of all users and brands.

Note that these embeddings are **NOT** the nn.Embeddings objects. Rather, we want to get the vectors which originate after running our HeteroData through our GraphSAGE network. We treat these vectors as final, trained user and brand embeddings.

In [ ]:
batch_size = 128

train_loader_for_preds = LinkNeighborLoader(
    data=train_data,
    num_neighbors=[20, 10],
    neg_sampling_ratio=0.0,
    edge_label_index=(("user", "buys", "brand"), edge_label_index),
    edge_label=edge_label,
    batch_size=batch_size,
    shuffle=False,
)

In [ ]:
from sklearn.metrics import roc_auc_score

user_ids_set = set()
user_ids = []
user_emb = []

brand_ids_set = set()
brand_ids = []
brand_emb = []

for sampled_data in tqdm.tqdm(train_loader_for_preds):
    with torch.no_grad():
        sampled_data.to(device)
        x_dict = model.encode(sampled_data)

        u_ids = sampled_data["user"].node_id
        b_ids = sampled_data["brand"].node_id

        user_embeddings = x_dict["user"]
        brand_embeddings = x_dict["brand"]

        for u_id, u_emb in zip(u_ids, user_embeddings):
            if u_id not in user_ids_set:
                user_ids.append(u_id)
                user_emb.append(u_emb)
                user_ids_set.add(u_id)

        for b_id, b_emb in zip(b_ids, brand_embeddings):
            if b_id not in brand_ids_set:
                brand_ids.append(b_id)
                brand_emb.append(b_emb)
                brand_ids_set.add(b_id)


In [ ]:
target_brand_embeddings = [brand_emb[brand_ids.index(b_id)] for b_id in target_brand_ids]

In [ ]:
target_brand_embeddings = torch.stack(target_brand_embeddings).cpu()
user_embeddings = torch.stack(user_emb).cpu()

In [ ]:
user_ids = torch.tensor(user_ids, device = 'cpu')

# Get the Final Predictions

To get user and brand "relatedness" scores, we perform a dot product - similarly as during training. A high dot product value for a (user, brand) pair signifies a high relatedness.

In [ ]:
preds = torch.matmul(user_embeddings, target_brand_embeddings.T.cpu()).sigmoid()

In [ ]:
ground_truths = targets[user_ids]

In [ ]:
auc = roc_auc_score(ground_truths, preds)
print(f"Validation AUC: {auc:.4f}")

# Exercise 2

1. Change the neighborhood size in sampling to:
* **10** first-order neighbors and **10** second-order neighbor,
* **10** first-order neighbors, **10** second-order neighbors, **10** third-order neighbors, **10** fourth-order neighbors.

For each configuration, start training the network and look at the estimated training time. You do not need to finish training, just look at how long the training is expected to take.
Also, look at the network structure and see whether it changes between runs.

**Q1:** Does the network structure change for the 2 configurations?

**Q2:** How does training time change? Why does it change if network structure and batch size are the same?

**Q3:** Does the estimated time keep even or suddenly jumps in the tqdm bar?



In [ ]:
#@title Solution

train_loader = LinkNeighborLoader(
    data=train_data,
    num_neighbors=[10, 10],
    neg_sampling_ratio=2.0,
    edge_label_index=(("user", "buys", "brand"), edge_label_index),
    edge_label=edge_label,
    batch_size=128,
    shuffle=True,
)

train_loader = LinkNeighborLoader(
    data=train_data,
    num_neighbors=[10, 10, 10, 10],
    neg_sampling_ratio=2.0,
    edge_label_index=(("user", "buys", "brand"), edge_label_index),
    edge_label=edge_label,
    batch_size=128,
    shuffle=True,
)

# Exercise 3

Add an extra layer of SAGEConv (in total, you want to have 3 SAGEConv layers). Train the network and inspect the results.

In [ ]:
#@title Solution

class GNN(torch.nn.Module):
    def __init__(self, hidden_channels):
        """
        Defines the architecture of our GNN, such as the type and number of layers.
        """
        super().__init__()
        #create 2 GraphSAGE layers:
        self.conv1 = SAGEConv(hidden_channels, hidden_channels)
        self.conv2 = SAGEConv(hidden_channels, hidden_channels)
        self.conv3 = SAGEConv(hidden_channels, hidden_channels)
    def forward(self, x: Tensor, edge_index: Tensor) -> Tensor:
        x = F.relu(self.conv1(x, edge_index))
        x = F.relu(self.conv2(x, edge_index))
        x = self.conv3(x, edge_index)
        return x